In [1]:
from forse.networks.dcgan import *
import matplotlib.pyplot as plt
from forse.tools import *

Using TensorFlow backend.


In [2]:
dcgan = DCGAN(output_directory='/global/homes/k/krach/scratch/NNforFG/DCGAN/tests', img_size=(64, 64))

In [3]:
training_path = '/global/homes/k/krach/scratch/NNforFG/training_set/'
training_file = 'training_set_1000patches_20x20deg_T_HR1deg_LR5deg_Npix64_set2.npy'
patch_file = training_path+training_file
dcgan.train(epochs=10, patches_file=patch_file, batch_size=32, save_interval=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training Data Shape:  (800, 64, 64, 1)
Instructions for updating:
Use tf.cast instead.


/usr/common/software/tensorflow/gpu-tensorflow/1.13.1-py36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 2.3365237712860107 | D Accuracy: 31.25] [G loss: 0.8198785781860352]
1 [D loss: 0.7118649482727051 | D Accuracy: 59.375] [G loss: 0.6959409713745117]
2 [D loss: 0.9929590821266174 | D Accuracy: 37.5] [G loss: 3.060602903366089]
3 [D loss: 1.354258418083191 | D Accuracy: 34.375] [G loss: 1.8091731071472168]
4 [D loss: 0.701034665107727 | D Accuracy: 71.875] [G loss: 1.6853151321411133]
5 [D loss: 0.8772628903388977 | D Accuracy: 53.125] [G loss: 2.8245110511779785]
6 [D loss: 0.17480626702308655 | D Accuracy: 100.0] [G loss: 2.2451930046081543]
7 [D loss: 0.9335829019546509 | D Accuracy: 50.0] [G loss: 2.0048656463623047]
8 [D loss: 0.8353530168533325 | D Accuracy: 62.5] [G loss: 3.1724064350128174]
9 [D loss: 1.1839549541473389 | D Accuracy: 40.625] [G loss: 4.280172348022461]
